In [140]:
import time
notebookstart= time.time()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier
# Gradient Boosting
import lightgbm as lgb
import xgboost as xgb
import category_encoders as ce
# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor 
# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.cluster.vq import kmeans2, whiten
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from catboost import CatBoostRegressor
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



In [141]:
num_rows = None
EPS = 1e-100

In [142]:
train = pd.read_csv('/media/limbo/Home-Credit/data/application_train.csv.zip')
y = train['TARGET']

n_train = train.shape[0]

In [143]:
descretize = lambda x, n: list(map(str, list(pd.qcut(x, n, duplicates='drop'))))

def binary_encoder(df, n_train):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    enc = ce.BinaryEncoder(impute_missing=True, cols=categorical_columns).fit(df[0:n_train], df[0:n_train]['TARGET'])
    df = enc.transform(df)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df[new_columns]

In [144]:
def application_train_test(num_rows=num_rows, nan_as_category=False):
    # Read data and merge
    df = pd.read_csv('../data/application_train.csv.zip', nrows=num_rows)

    n_train = df.shape[0]

    test_df = pd.read_csv('../data/application_test.csv.zip', nrows=num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()

    df['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
    df['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)

    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']

    docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
    live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

    inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

    df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['NEW_AMT_INCOME_TOTAL_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['NEW_DOC_IND_AVG'] = df[docs].mean(axis=1)
    df['NEW_DOC_IND_STD'] = df[docs].std(axis=1)
    df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
    df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
    df['NEW_LIVE_IND_STD'] = df[live].std(axis=1)
    df['NEW_LIVE_IND_KURT'] = df[live].kurtosis(axis=1)
    df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
    df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
    df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
    df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
    df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    df['NEW_PHONE_TO_EMPLOY_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

#     df['children_ratio'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']

#     df['NEW_EXT_SOURCES_MEDIAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1)

#     df['NEW_DOC_IND_SKEW'] = df[docs].skew(axis=1)
#     df['NEW_LIVE_IND_SKEW'] = df[live].skew(axis=1)

#     df['ind_0'] = df['DAYS_EMPLOYED'] - df['DAYS_EMPLOYED'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['DAYS_EMPLOYED'].dropna().median()).mean()
#     df['ind_1'] = df['DAYS_EMPLOYED'] - df['DAYS_EMPLOYED'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['DAYS_EMPLOYED'].dropna().median()).median()

#     df['ind_2'] = df['DAYS_BIRTH'] - df['DAYS_BIRTH'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['DAYS_BIRTH'].dropna().median()).mean()
#     df['ind_3'] = df['DAYS_BIRTH'] - df['DAYS_BIRTH'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['DAYS_BIRTH'].dropna().median()).median()

#     df['ind_4'] = df['AMT_INCOME_TOTAL'] - df['AMT_INCOME_TOTAL'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_INCOME_TOTAL'].dropna().median()).mean()
#     df['ind_5'] = df['AMT_INCOME_TOTAL'] - df['AMT_INCOME_TOTAL'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_INCOME_TOTAL'].dropna().median()).median()

#     df['ind_6'] = df['AMT_CREDIT'] - df['AMT_CREDIT'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_CREDIT'].dropna().median()).mean()
#     df['ind_7'] = df['AMT_CREDIT'] - df['AMT_CREDIT'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_CREDIT'].dropna().median()).median()

#     df['ind_8'] = df['AMT_ANNUITY'] - df['AMT_ANNUITY'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_ANNUITY'].dropna().median()).mean()
#     df['ind_9'] = df['AMT_ANNUITY'] - df['AMT_ANNUITY'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_ANNUITY'].dropna().median()).median()

#     df['ind_10'] = df['AMT_CREDIT'] - df['AMT_INCOME_TOTAL'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_INCOME_TOTAL'].dropna().median()).mean()
#     df['ind_11'] = df['AMT_CREDIT'] - df['AMT_INCOME_TOTAL'].replace([np.inf, -np.inf], np.nan).fillna(
#         df['AMT_INCOME_TOTAL'].dropna().median()).median()

#     AGGREGATION_RECIPIES = [
#         (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
#                                                   ('AMT_CREDIT', 'max'),
#                                                   ('EXT_SOURCE_1', 'mean'),
#                                                   ('EXT_SOURCE_2', 'mean'),
#                                                   ('OWN_CAR_AGE', 'max'),
#                                                   ('OWN_CAR_AGE', 'sum')]),
#         (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
#                                                 ('AMT_INCOME_TOTAL', 'mean'),
#                                                 ('DAYS_REGISTRATION', 'mean'),
#                                                 ('EXT_SOURCE_1', 'mean'),
#                                                 ('NEW_CREDIT_TO_ANNUITY_RATIO', 'mean')]),
#         (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
#                                                      ('CNT_CHILDREN', 'mean'),
#                                                      ('DAYS_ID_PUBLISH', 'mean')]),
#         (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('EXT_SOURCE_1', 'mean'),
#                                                                                                ('EXT_SOURCE_2',
#                                                                                                 'mean')]),
#         (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
#                                                       ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
#                                                       ('APARTMENTS_AVG', 'mean'),
#                                                       ('BASEMENTAREA_AVG', 'mean'),
#                                                       ('EXT_SOURCE_1', 'mean'),
#                                                       ('EXT_SOURCE_2', 'mean'),
#                                                       ('EXT_SOURCE_3', 'mean'),
#                                                       ('NONLIVINGAREA_AVG', 'mean'),
#                                                       ('OWN_CAR_AGE', 'mean')]),
#         (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean'),
#                                                                                 ('EXT_SOURCE_1', 'mean')]),
#         (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
#                                ('CNT_CHILDREN', 'mean'),
#                                ('CNT_FAM_MEMBERS',  'mean'),
#                                ('DAYS_BIRTH', 'mean'),
#                                ('DAYS_EMPLOYED',  'mean'),
#                                ('NEW_CREDIT_TO_ANNUITY_RATIO', 'median'),
#                                ('DAYS_REGISTRATION', 'mean'),
#                                ('EXT_SOURCE_1', 'mean'),
#                                ('EXT_SOURCE_2', 'mean'),
#                                ('EXT_SOURCE_3',  'mean')]),
#     ]

#     for groupby_cols, specs in AGGREGATION_RECIPIES:
#         group_object = df.groupby(groupby_cols)
#         for select, agg in specs:
#             groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
#             df = df.merge(group_object[select]
#                           .agg(agg)
#                           .reset_index()
#                           .rename(index=str,
#                                   columns={select: groupby_aggregate_name})
#                           [groupby_cols + [groupby_aggregate_name]],
#                           on=groupby_cols,
#                           how='left')
    # ['DAYS_EMPLOYED', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN', 'credit_per_person', 'cnt_non_child']
    df['retirement_age'] = (df['DAYS_BIRTH'] > -14000).astype(int)
    df['long_employment'] = (df['DAYS_EMPLOYED'] > -2000).astype(int)
    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['child_to_non_child_ratio'] = df['CNT_CHILDREN'] / df['cnt_non_child']
    df['income_per_non_child'] = df['AMT_INCOME_TOTAL'] / df['cnt_non_child']
    df['credit_per_person'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']
    df['credit_per_child'] = df['AMT_CREDIT'] / (1 + df['CNT_CHILDREN'])
    df['credit_per_non_child'] = df['AMT_CREDIT'] / df['cnt_non_child']

    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['child_to_non_child_ratio'] = df['CNT_CHILDREN'] / df['cnt_non_child']
    df['income_per_non_child'] = df['AMT_INCOME_TOTAL'] / df['cnt_non_child']
    df['credit_per_person'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']
    df['credit_per_child'] = df['AMT_CREDIT'] / (1 + df['CNT_CHILDREN'])
    df['credit_per_non_child'] = df['AMT_CREDIT'] / df['cnt_non_child']

#     df['p_0'] = descretize(df['credit_per_non_child'].values, 2 ** 5)
#     df['p_1'] = descretize(df['credit_per_person'].values, 2 ** 5)
#     df['p_2'] = descretize(df['credit_per_child'].values, 2 ** 5)
#     df['p_3'] = descretize(df['retirement_age'].values, 2 ** 5)
#     df['p_4'] = descretize(df['income_per_non_child'].values, 2 ** 5)
#     df['p_5'] = descretize(df['child_to_non_child_ratio'].values, 2 ** 5)

#     df['p_6'] = descretize(df['NEW_CREDIT_TO_ANNUITY_RATIO'].values, 2 ** 5)
#     df['p_7'] = descretize(df['NEW_CREDIT_TO_ANNUITY_RATIO'].values, 2 ** 6)
#     df['p_8'] = descretize(df['NEW_CREDIT_TO_ANNUITY_RATIO'].values, 2 ** 7)

#     df['pe_0'] = descretize(df['credit_per_non_child'].values, 2 ** 6)
#     df['pe_1'] = descretize(df['credit_per_person'].values, 2 ** 6)
#     df['pe_2'] = descretize(df['credit_per_child'].values, 2 ** 6)
#     df['pe_3'] = descretize(df['retirement_age'].values, 2 ** 6)
#     df['pe_4'] = descretize(df['income_per_non_child'].values, 2 ** 6)
#     df['pe_5'] = descretize(df['child_to_non_child_ratio'].values, 2 ** 6)

    c = df['NEW_CREDIT_TO_ANNUITY_RATIO'].replace([np.inf, -np.inf], np.nan).fillna(999).values
    a, b = kmeans2(np.log1p(c), 2, iter=333)
    df['x_0'] = b

    a, b = kmeans2(np.log1p(c), 4, iter=333)
    df['x_1'] = b

    a, b = kmeans2(np.log1p(c), 8, iter=333)
    df['x_2'] = b

    a, b = kmeans2(np.log1p(c), 16, iter=333)
    df['x_3'] = b

    a, b = kmeans2(np.log1p(c), 32, iter=333)
    df['x_4'] = b

    a, b = kmeans2(np.log1p(c), 64, iter=333)
    df['x_5'] = b

    a, b = kmeans2(np.log1p(c), 128, iter=333)
    df['x_6'] = b

    a, b = kmeans2(np.log1p(c), 150, iter=333)
    df['x_7'] = b

    a, b = kmeans2(np.log1p(c), 256, iter=333)
    df['x_8'] = b

    a, b = kmeans2(np.log1p(c), 512, iter=333)
    df['x_9'] = b

    a, b = kmeans2(np.log1p(c), 1024, iter=333)
    df['x_10'] = b
    
    
#     c = df['EXT_SOURCE_1'].replace([np.inf, -np.inf], np.nan).fillna(999).values
#     a, b = kmeans2(np.log1p(c), 2, iter=333)
#     df['ex1_0'] = b

#     a, b = kmeans2(np.log1p(c), 4, iter=333)
#     df['ex1_1'] = b

#     a, b = kmeans2(np.log1p(c), 8, iter=333)
#     df['ex1_2'] = b

#     a, b = kmeans2(np.log1p(c), 16, iter=333)
#     df['ex1_3'] = b

#     a, b = kmeans2(np.log1p(c), 32, iter=333)
#     df['ex1_4'] = b

#     a, b = kmeans2(np.log1p(c), 64, iter=333)
#     df['ex1_5'] = b

#     a, b = kmeans2(np.log1p(c), 128, iter=333)
#     df['ex1_6'] = b

#     a, b = kmeans2(np.log1p(c), 256, iter=333)
#     df['ex1_7'] = b    
    
    
#     c = df['EXT_SOURCE_2'].replace([np.inf, -np.inf], np.nan).fillna(999).values
#     a, b = kmeans2(np.log1p(c), 2, iter=333)
#     df['ex2_0'] = b

#     a, b = kmeans2(np.log1p(c), 4, iter=333)
#     df['ex2_1'] = b

#     a, b = kmeans2(np.log1p(c), 8, iter=333)
#     df['ex2_2'] = b

#     a, b = kmeans2(np.log1p(c), 16, iter=333)
#     df['ex2_3'] = b

#     a, b = kmeans2(np.log1p(c), 32, iter=333)
#     df['ex2_4'] = b

#     a, b = kmeans2(np.log1p(c), 64, iter=333)
#     df['ex2_5'] = b

#     a, b = kmeans2(np.log1p(c), 128, iter=333)
#     df['ex2_6'] = b

#     a, b = kmeans2(np.log1p(c), 256, iter=333)
#     df['ex2_7'] = b 
    
#     c = df['EXT_SOURCE_3'].replace([np.inf, -np.inf], np.nan).fillna(999).values
#     a, b = kmeans2(np.log1p(c), 2, iter=333)
#     df['ex3_0'] = b

#     a, b = kmeans2(np.log1p(c), 4, iter=333)
#     df['ex3_1'] = b

#     a, b = kmeans2(np.log1p(c), 8, iter=333)
#     df['ex3_2'] = b

#     a, b = kmeans2(np.log1p(c), 16, iter=333)
#     df['ex3_3'] = b

#     a, b = kmeans2(np.log1p(c), 32, iter=333)
#     df['ex3_4'] = b

#     a, b = kmeans2(np.log1p(c), 64, iter=333)
#     df['ex3_5'] = b

#     a, b = kmeans2(np.log1p(c), 128, iter=333)
#     df['ex3_6'] = b

#     a, b = kmeans2(np.log1p(c), 256, iter=333)
#     df['ex3_7'] = b 
    

#     df['ex_1_0'] = descretize(df['EXT_SOURCE_1'].values, 2 ** 6)
#     df['ex_2_0'] = descretize(df['EXT_SOURCE_2'].values, 2 ** 6)
#     df['ex_3_0'] = descretize(df['EXT_SOURCE_3'].values, 2 ** 6)

#     df['ex_1_1'] = descretize(df['EXT_SOURCE_1'].values, 2 ** 4)
#     df['ex_2_1'] = descretize(df['EXT_SOURCE_2'].values, 2 ** 4)
#     df['ex_3_1'] = descretize(df['EXT_SOURCE_3'].values, 2 ** 4)

#     df['ex_1_2'] = descretize(df['EXT_SOURCE_1'].values, 2 ** 5)
#     df['ex_2_2'] = descretize(df['EXT_SOURCE_2'].values, 2 ** 5)
#     df['ex_3_2'] = descretize(df['EXT_SOURCE_3'].values, 2 ** 5)

#     df['ex_1_3'] = descretize(df['EXT_SOURCE_1'].values, 2 ** 3)
#     df['ex_2_4'] = descretize(df['EXT_SOURCE_2'].values, 2 ** 3)
#     df['ex_3_5'] = descretize(df['EXT_SOURCE_3'].values, 2 ** 3)

#     c = df['NEW_EXT_SOURCES_MEAN'].replace([np.inf, -np.inf], np.nan).fillna(999).values
#     a, b = kmeans2(np.log1p(c), 2, iter=333)
#     df['ex_mean_0'] = b

#     a, b = kmeans2(np.log1p(c), 4, iter=333)
#     df['ex_mean_1'] = b

#     a, b = kmeans2(np.log1p(c), 8, iter=333)
#     df['ex_mean_2'] = b

#     a, b = kmeans2(np.log1p(c), 16, iter=333)
#     df['ex_mean_3'] = b

#     a, b = kmeans2(np.log1p(c), 32, iter=333)
#     df['ex_mean_4'] = b

#     a, b = kmeans2(np.log1p(c), 64, iter=333)
#     df['ex_mean_5'] = b
    
    
    


#     df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    
#     df['ex1/ex2'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_2']
#     df['ex1/ex3'] = df['EXT_SOURCE_1'] / df['EXT_SOURCE_3']
#     df['ex2/ex3'] = df['EXT_SOURCE_3'] / df['EXT_SOURCE_3']
    
#     df['ex1*ex2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
#     df['ex1*ex3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
#     df['ex2*ex3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    
#     df['cred*ex1'] = df['AMT_CREDIT'] * df['EXT_SOURCE_1']
#     df['cred*ex2'] = df['AMT_CREDIT'] * df['EXT_SOURCE_2']
#     df['cred*ex3'] = df['AMT_CREDIT'] * df['EXT_SOURCE_3']
    
#     df['cred/ex1'] = df['AMT_CREDIT'] / df['EXT_SOURCE_1']
#     df['cred/ex2'] = df['AMT_CREDIT'] / df['EXT_SOURCE_2']
#     df['cred/ex3'] = df['AMT_CREDIT'] / df['EXT_SOURCE_3']
    
#     df['cred*ex123'] = df['AMT_CREDIT'] * df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    
#     del df['EXT_SOURCE_1']
#     del df['EXT_SOURCE_2']
#     del df['EXT_SOURCE_3']
#     del df['NEW_EXT_SOURCES_MEAN']

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    del test_df
    gc.collect()
    return df

In [145]:
df = application_train_test(num_rows=num_rows, nan_as_category=False)

Train samples: 307511, test samples: 48744


/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/scipy/cluster/vq.py:523: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


In [146]:
df.head()

,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,...,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
0,0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,...,2,5,12,3,31,8,42,157,277,106
1,1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,...,3,6,8,22,42,68,81,112,98,531
2,2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,...,2,4,7,15,2,56,73,207,120,289
3,3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,...,0,7,9,20,10,7,25,217,351,74
4,4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,...,1,2,6,10,60,16,141,172,264,147


In [204]:
selected_features = ['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'NEW_CREDIT_TO_ANNUITY_RATIO', 'NEW_CREDIT_TO_GOODS_RATIO', 'NEW_CREDIT_TO_INCOME_RATIO']  + ['x_' + str(x) for x in range(11)] + \
['retirement_age', 'long_employment'] + ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']



In [147]:
categorical_columns = [col for col in train.columns if train[col].dtype == 'object']

In [148]:
numerical_columns = [col for col in df.columns if df[col].dtype != 'object']

In [149]:
new_df = df.copy()

In [150]:
df = new_df

In [151]:
encoder = preprocessing.LabelEncoder()

for f in categorical_columns:
    if df[f].dtype == 'object':
        df[f] = encoder.fit_transform(df[f].apply(str).values) 

In [152]:
categorical_columns

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [153]:
gc.collect()

21

In [154]:
train = pd.read_csv('../data/application_train.csv.zip', nrows=num_rows)
n_train = train.shape[0]
test = pd.read_csv('../data/application_test.csv.zip', nrows=num_rows)
    
new_df = pd.concat([train, test], axis=0)
gc.collect()

/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


28

In [155]:
new_df.shape

(356255, 122)

In [156]:
new_df[categorical_columns].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN


In [157]:
encoder = preprocessing.LabelEncoder()

for f in categorical_columns:
    if new_df[f].dtype == 'object':
        new_df[f] = encoder.fit_transform(new_df[f].apply(str).values) 

In [158]:
new_features = pd.read_csv('selected_features.csv', header=0, index_col=None)

In [159]:
new_features.head()

,EXT_1_DIV_BIRTH_SURESH,EXT_3_DIV_BIRTH_SURESH,EXT_2_DIV_BIRTH_SURESH,INSTALLMENT_PAID_LATE_LAST_LOAN_MEAN_SURESH,AMT_CREDIT_DIFF_AMT_GOODS_SURESH,BUREAU_DEBT_CREDIT_RATIO_SURESH,PREVIOUS_APPLICATION_FRACTION_OF_REFUSED_APPLICATIONS_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_MAX_SURESH,TREND_INTER_FUTURE_CNT_SURESH,TREND_COEF_DPD_DEF_SURESH,...,INSTALLMENT_PAID_LATE_IN_DAYS_12_MONTHS_MAX_SURESH,BUREAU_CREDIT_ACTIVE_BINARY_MEAN_SURESH,DEF_30_60_SOCIAL_SUM_SURESH,POPULAR_AMT_GOODS_PRICE_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_MEAN_SURESH,MEAN_PAYMENT_CC_SURESH,DAYS_EMPLOYED_DIFF_DAYS_BIRTH_SURESH,TREND_COEF_PAID_LATE_SURESH,AMT_INCOME_TOTAL_12_AMT_ANNUITY_SURESH,EXT_SOURCE_3
0,-0.000009,-0.000015,-0.000028,0.0,55597.5,NaN,0.000000,-12.0,5.000000,0.0,...,-12.0,0.25,4,0,-20.421053,NaN,8824.0,0.157343,-7825.5,0.139376
1,-0.000019,NaN,-0.000037,0.0,164002.5,0.0,0.000000,-3.0,0.000000,0.0,...,NaN,0.25,0,0,-4.428571,NaN,15577.0,NaN,-13198.5,NaN
2,NaN,-0.000038,-0.000029,0.0,0.0,0.0,0.000000,-3.0,NaN,0.0,...,NaN,0.00,0,0,-7.666667,NaN,18821.0,NaN,-1125.0,0.729567
3,NaN,NaN,-0.000034,0.0,15682.5,NaN,0.111111,-1.0,7.919033,0.0,...,-1.0,NaN,0,0,-4.500000,0.0,15966.0,1.181081,-18436.5,NaN
4,NaN,NaN,-0.000016,0.0,0.0,0.0,0.000000,0.0,20.205607,0.0,...,0.0,0.00,0,0,-2.250000,NaN,16894.0,-0.504714,-11740.5,NaN


In [205]:
my_features = [f for f in selected_features if f not in new_features.columns]

In [206]:
my_features

['AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_INCOME_TOTAL',
 'NEW_CREDIT_TO_ANNUITY_RATIO',
 'NEW_CREDIT_TO_GOODS_RATIO',
 'NEW_CREDIT_TO_INCOME_RATIO',
 'x_0',
 'x_1',
 'x_2',
 'x_3',
 'x_4',
 'x_5',
 'x_6',
 'x_7',
 'x_8',
 'x_9',
 'x_10',
 'retirement_age',
 'long_employment',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2']

In [203]:
new_df[categorical_columns][0:n_train].shape

(307511, 16)

In [163]:
new_df[categorical_columns][n_train:].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,0,0,0,1,6,7,1,1,1,18,5,28,0,0,5,0
1,0,1,0,1,6,7,4,1,1,9,0,42,0,1,7,2
2,0,1,1,1,7,7,1,1,1,4,1,54,0,1,7,2
3,0,0,0,1,6,7,4,1,1,14,6,5,3,0,4,0
4,0,1,1,0,6,7,4,1,1,18,0,5,0,1,7,2


In [164]:
suresh_august16 = pd.read_csv('../data/SureshFeaturesAug16.csv', header=0, index_col=None)


In [165]:
suresh_august16.head()

,CUSTOMER_EXT1_OVERREGION_SURESH,CUSTOMER_EXT2_OVERREGION_SURESH,CUSTOMER_EXT3_OVERREGION_SURESH,EXTERNAL_SOURCE_1_MEDIAN_SURESH,EXTERNAL_SOURCE_2_MEDIAN_SURESH,EXTERNAL_SOURCE_3_MEDIAN_SURESH,REGION_POPULATION_RELATIVE_CAT_SURESH,SK_ID_CURR
0,5.803887,1.889160,3.659815,0.481937,0.496752,0.510090,65,100002
1,1.684089,0.946854,inf,0.524202,0.589176,0.506484,13,100003
2,inf,0.917455,0.767969,0.462446,0.510024,0.560284,49,100004
3,inf,0.826746,inf,0.503594,0.537750,0.506484,36,100006
4,inf,1.695639,inf,0.523748,0.547248,0.517297,76,100007


In [166]:
del suresh_august16['SK_ID_CURR']

In [167]:
goran_features = pd.read_csv('../goran-data/goranm_feats_v3.csv', header=0, index_col=None)
goran_features.head()

,AMT_GOODS_PRICE_TO_CREDIT,AMT_CREDIT_INSTALLMENTS,AMT_CREDIT_INTEREST_MAYBE,AMT_CREDIT_INTEREST_MAYBE_RAT,AMT_INCOME_TOTAL_RAT_CREDIT,NAME_CONTRACT_TYPE_Consumer_loans,NAME_CONTRACT_TYPE_Cash_loans,NAME_CONTRACT_TYPE_Revolving_loans,NAME_CONTRACT_TYPE_XNA,PRE_RECORD_COUNT,...,STCK_PAY_480_.,STCK_CC_6_.,STCK_BERBAL_6_.,EXT_MEAN,TERM_BEFORE_END,DAYS_DECISION_MEAN,AMT_CREDIT_LIMIT_ACTUAL_6,AMT_CREDIT_LIMIT_ACTUAL_12,SK_ID_CURR,IS_TRAIN
0,0.863262,16.461104,11389.5,0.028012,0.144444,0.0,1.0,0.0,0.0,1.0,...,0.080736,NaN,0.160535,0.161787,1.0,-606.0,NaN,NaN,100002,1
1,0.873211,36.234085,8356.5,0.006460,0.348611,1.0,2.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,3.0,-3915.0,NaN,NaN,100003,1
2,1.000000,20.000000,0.0,0.000000,0.166667,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,1.0,-815.0,NaN,NaN,100004,1
3,0.949845,10.532818,15817.5,0.050586,0.183333,5.0,2.0,2.0,0.0,9.0,...,0.073751,0.03353,NaN,NaN,2.0,-2452.0,270000.0,270000.0,100006,1
4,1.000000,23.461618,10093.5,0.019675,0.351852,4.0,2.0,0.0,0.0,6.0,...,0.076197,NaN,NaN,NaN,4.0,-7337.0,NaN,NaN,100007,1


In [168]:
del goran_features['SK_ID_CURR']
del goran_features['IS_TRAIN']

In [169]:
goran_features_19_8 = pd.read_csv('../data/goranm_feats_19_08.csv', header=0, index_col=None)

In [170]:
goran_features_19_8.head()

,AMT_INCOME_TO_REGION_MEAN,EXT_3_TO_REGION_MEAN,EXT_2_TO_REGION_MEAN,EXT_1_TO_REGION_MEAN,AMT_CREDIT_INSTALLMENTS_TO_REGION_MEAN,AMT_INCOME_TOTAL_RAT_CREDIT_TO_REGION_MEAN,MEAN_CREDIT_DURATION_TO_REGION_MEAN,MAX_DAYS_LAST_DUE_TO_REGION_MEAN,SK_ID_CURR
0,1.144653,0.286892,0.591183,0.171547,0.769676,0.505468,0.403411,-0.000140,100002
1,1.124447,NaN,1.152353,0.605032,1.554841,1.221852,0.441247,-0.004194,100003
2,0.357478,1.364991,1.188570,NaN,0.982184,0.637138,0.203086,-0.004140,100004
3,0.739426,NaN,1.266580,NaN,0.494683,0.697055,NaN,1.983986,100006
4,0.795062,NaN,0.641314,NaN,1.132934,1.132543,0.185846,2.215766,100007


In [171]:
del goran_features_19_8['SK_ID_CURR']

In [172]:
from sklearn.externals import joblib

prevs_df = joblib.load('../data/prev_application_solution3_v2')


In [173]:
prevs_df.head()

,SK_ID_CURR,prev_application_solution3_v2
0,100002,0.115430
1,100003,0.019078
2,100004,0.084967
3,100006,0.123526
4,100007,0.098192


In [174]:
suresh_august16_2 = pd.read_csv('../data/SureshFeaturesAug16_2.csv', header=0, index_col=None)
suresh_august15 = pd.read_csv('../data/SureshFeaturesAug15.csv', header=0, index_col=None)
suresh_august16 = pd.read_csv('../data/SureshFeaturesAug16.csv', header=0, index_col=None)
suresh_august19 = pd.read_csv('../data/suresh_features_Aug19th.csv', header=0, index_col=None)
suresh_august19_2 = pd.read_csv('../data/SureshFeatures_19_2th.csv', header=0, index_col=None)

In [175]:
suresh_august20 = pd.read_csv('../data/SureshFeatures3BestAgu20.csv', header=0, index_col=None)

In [176]:
suresh_august20.head(100)

,SK_ID_CURR,PAYMENT_FEATURE_1_SURESH,PAYMENT_FEATURE_2_SURESH,PAYMENT_FEATURE_3_SURESH
0,100002,NaN,NaN,NaN
1,100003,NaN,NaN,NaN
2,100004,NaN,NaN,NaN
3,100006,NaN,NaN,NaN
4,100007,NaN,NaN,NaN
5,100008,NaN,NaN,NaN
6,100009,NaN,NaN,NaN
7,100010,NaN,NaN,NaN
8,100011,2.384618,149254.425,74627.2125
9,100012,0.399495,4417.425,4417.4250


In [177]:
del suresh_august15['SK_ID_CURR']
del suresh_august16_2['SK_ID_CURR']
del suresh_august19['SK_ID_CURR_SURESH']
del suresh_august16['SK_ID_CURR']
del suresh_august19_2['SK_ID_CURR']
suresh_august15.head()

,NUM_ACTIVE_CASH_LOANS_SURESH,NO_ACTIVE_CC_LOANS_SURESH,FUTURE_CASH_INSTALLMENTS_SUM_SURESH,MEAN_DPD_6MONTHS_CASH_LOANS_SURESH,TOTAL_PREV_CASH_LOANS_SURESH,TOTAL_PREV_REVO_LOANS_SURESH,PER_X_SELL_LOANS_SURESH,TOTAL_WALK_IN_LOANS_SURESH,PER_WALK_IN_LOANS_SURESH,TOTAL_CC_PAYMENTS_SURESH,...,AMT_INCOME_TOTAL_12_AMT_ANNUITY_2_SURESH,TOTAL_CC_LOADING_6MONTHS_SURESH,INSTALLMENT_PAID_LATE_IN_DAYS_12_MONTHS_SUM_SURESH,COUNT_SCOFR_SURESH,INSTALLMENT_PAID_OVER_AMOUNT_LAST_LOAN_MEAN_SURESH,CREDIT_PER_NON_CHILD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SUM_SURESH,MEAN_PAYMENT_CC_SURESH,DAYS_DECISION_MAX_SURESH
0,1.0,NaN,6.0,0.0,NaN,NaN,0.000000,0.0,0.000000,NaN,...,NaN,NaN,-211.0,NaN,0.0,406597.50,4.925171,-388.0,NaN,-606.0
1,3.0,NaN,NaN,NaN,1.0,NaN,0.333333,0.0,0.000000,NaN,...,NaN,NaN,NaN,NaN,0.0,646751.25,1.718249,-31.0,NaN,-746.0
2,1.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.000000,NaN,...,NaN,NaN,NaN,NaN,0.0,135000.00,4.163332,-23.0,NaN,-815.0
3,3.0,1.0,3.0,0.0,2.0,1.0,0.600000,0.0,0.000000,0.0,...,-18436.5,0.0,-68.0,NaN,0.0,156341.25,3.628590,-45.0,0.0,-181.0
4,5.0,NaN,13.0,0.0,4.0,NaN,0.500000,1.0,0.166667,NaN,...,NaN,NaN,-47.0,NaN,0.0,513000.00,1.422226,-27.0,NaN,-374.0


In [178]:
suresh_20 = pd.read_csv('../data/SureshFeatures20_2.csv', header=0, index_col=None)

In [179]:

suresh_20.head(100)


,SK_ID_CURR,PAYMENT_FEATURE_4_SURESH,PAYMENT_FEATURE_5_SURESH,PAYMENT_FEATURE_6_SURESH
0,100002,NaN,NaN,NaN
1,100003,NaN,NaN,NaN
2,100004,NaN,NaN,NaN
3,100006,NaN,NaN,NaN
4,100007,NaN,NaN,NaN
5,100008,NaN,NaN,NaN
6,100009,NaN,NaN,NaN
7,100010,NaN,NaN,NaN
8,100011,NaN,NaN,NaN
9,100012,NaN,NaN,NaN


In [180]:
del suresh_20['SK_ID_CURR']

In [181]:
goranm_8_20 = pd.read_csv('../data/goranm_08_20.csv', header=0, index_col=None)


In [182]:
goranm_8_20.head()

,MAYBE_START_OF_APPLICATION,DAYS_CURRENT_END,AGE_AT_END_OF_LOAND,MEAN_OVERDUE_CORRECTED,SK_ID_CURR
0,-606.0,-112.166879,25.920548,150.0,100002
1,-291.0,796.022564,48.112391,50.0,100003
2,-225.0,375.000000,53.208219,30.0,100004
3,-181.0,134.984538,52.438314,NaN,100006
4,-374.0,329.848529,55.511914,NaN,100007


In [183]:
del goranm_8_20['SK_ID_CURR']

In [184]:
def do_countuniq( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Counting unqiue ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_mean(df, group_cols, counted, agg_name, agg_type='float32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Calculating mean of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_count(df, group_cols, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

In [185]:
counts_columns = []
for f_0 in categorical_columns:
    for f_1 in [x for x in categorical_columns if x != f_0] :
        df = do_countuniq(df, [f_0], f_1,
                      f_0 + '-' + f_1 + '_cunique', 'uint16', show_max=True); gc.collect()
        counts_columns.append(f_0 + '-' + f_1 + '_cunique')

Counting unqiue  CODE_GENDER  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-FLAG_OWN_CAR_cunique max value =  2
Counting unqiue  FLAG_OWN_REALTY  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-FLAG_OWN_REALTY_cunique max value =  2
Counting unqiue  NAME_TYPE_SUITE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_TYPE_SUITE_cunique max value =  8
Counting unqiue  NAME_INCOME_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_HOUSING_TYPE_cunique max value =  6
Counting u

NAME_TYPE_SUITE-WEEKDAY_APPR_PROCESS_START_cunique max value =  7
Counting unqiue  ORGANIZATION_TYPE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-ORGANIZATION_TYPE_cunique max value =  58
Counting unqiue  FONDKAPREMONT_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-FONDKAPREMONT_MODE_cunique max value =  5
Counting unqiue  HOUSETYPE_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-HOUSETYPE_MODE_cunique max value =  4
Counting unqiue  WALLSMATERIAL_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-WALLSMATERIAL_MODE_cunique max value =  8
Counting unqiue  EMERGENCYSTATE_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-EMERGENCYSTATE_MODE_cunique max value =  3
Counting unqiue  NAME_CONTRACT_TYPE  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-NAME_CONTRACT_TYPE_cunique max value =  2
Counting unqiue  CODE_GENDER  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-FLAG_OWN_CAR_

OCCUPATION_TYPE-NAME_CONTRACT_TYPE_cunique max value =  2
Counting unqiue  CODE_GENDER  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-FLAG_OWN_CAR_cunique max value =  2
Counting unqiue  FLAG_OWN_REALTY  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-FLAG_OWN_REALTY_cunique max value =  2
Counting unqiue  NAME_TYPE_SUITE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_TYPE_SUITE_cunique max value =  8
Counting unqiue  NAME_INCOME_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_HOUSING_TYPE_cunique max value =  6


HOUSETYPE_MODE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_HOUSING_TYPE_cunique max value =  6
Counting unqiue  OCCUPATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-OCCUPATION_TYPE_cunique max value =  19
Counting unqiue  WEEKDAY_APPR_PROCESS_START  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-WEEKDAY_APPR_PROCESS_START_cunique max value =  7
Counting unqiue  ORGANIZATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-ORGANIZATION_TYPE_cunique max value =  58
Counting unqiue  FONDKAPREMONT_MODE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-FONDKAPREMONT_MODE_cunique max value =  5
Counting unqiue  WALLSMATERIAL_MODE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-WALLSMATE

In [186]:
count_columns = []
for f_0 in categorical_columns:
        df = do_count(df, [f_0],
                      f_0  + '_count', 'uint16', show_max=True); gc.collect()
        count_columns.append(f_0  + '_count')

Aggregating by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE_count max value =  326537
Aggregating by  ['CODE_GENDER'] ...
CODE_GENDER_count max value =  235126
Aggregating by  ['FLAG_OWN_CAR'] ...
FLAG_OWN_CAR_count max value =  235235
Aggregating by  ['FLAG_OWN_REALTY'] ...
FLAG_OWN_REALTY_count max value =  246970
Aggregating by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE_count max value =  288253
Aggregating by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE_count max value =  183307
Aggregating by  ['NAME_EDUCATION_TYPE'] ...
NAME_EDUCATION_TYPE_count max value =  252379
Aggregating by  ['NAME_FAMILY_STATUS'] ...
NAME_FAMILY_STATUS_count max value =  228715
Aggregating by  ['NAME_HOUSING_TYPE'] ...
NAME_HOUSING_TYPE_count max value =  316513
Aggregating by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE_count max value =  111996
Aggregating by  ['WEEKDAY_APPR_PROCESS_START'] ...
WEEKDAY_APPR_PROCESS_START_count max value =  63652
Aggregating by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE_count

In [187]:
for f in ['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_INCOME_TOTAL'] + ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']:
    new_df[f] = new_df[f].replace([np.inf, -np.inf], np.nan).fillna(new_df[f].replace([np.inf, -np.inf], np.nan).dropna().median())

In [188]:
mean_columns = []
for f_0 in categorical_columns:
    for f_1 in ['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_INCOME_TOTAL'] + ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'] :
        new_df = do_mean(new_df, [f_0], f_1,
                      f_0 + '-' + f_1 + '_mean', 'uint16', show_max=True); gc.collect()
        mean_columns.append(f_0 + '-' + f_1 + '_mean')

Calculating mean of  AMT_ANNUITY  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-AMT_ANNUITY_mean max value =  28439.780537886978
Calculating mean of  AMT_CREDIT  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-AMT_CREDIT_mean max value =  611841.0087876718
Calculating mean of  AMT_INCOME_TOTAL  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-AMT_INCOME_TOTAL_mean max value =  170440.64937227633
Calculating mean of  EXT_SOURCE_1  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-EXT_SOURCE_1_mean max value =  0.5045938349042247
Calculating mean of  EXT_SOURCE_2  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-EXT_SOURCE_2_mean max value =  0.5238536075754979
Calculating mean of  EXT_SOURCE_3  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-EXT_SOURCE_3_mean max value =  0.5141439408185883
Calculating mean of  AMT_ANNUITY  by  ['CODE_GENDER'] ...
CODE_GENDER-AMT_ANNUITY_mean max value =  29000.096321981426
Calculating mean of  AMT_CREDIT  by  ['CODE_GENDER'] ...
CODE_GENDER-A

WEEKDAY_APPR_PROCESS_START-EXT_SOURCE_1_mean max value =  0.5060303636356729
Calculating mean of  EXT_SOURCE_2  by  ['WEEKDAY_APPR_PROCESS_START'] ...
WEEKDAY_APPR_PROCESS_START-EXT_SOURCE_2_mean max value =  0.5165372526798266
Calculating mean of  EXT_SOURCE_3  by  ['WEEKDAY_APPR_PROCESS_START'] ...
WEEKDAY_APPR_PROCESS_START-EXT_SOURCE_3_mean max value =  0.5159211285228568
Calculating mean of  AMT_ANNUITY  by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE-AMT_ANNUITY_mean max value =  33651.590782122905
Calculating mean of  AMT_CREDIT  by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE-AMT_CREDIT_mean max value =  753659.748603352
Calculating mean of  AMT_INCOME_TOTAL  by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE-AMT_INCOME_TOTAL_mean max value =  255180.16759776536
Calculating mean of  EXT_SOURCE_1  by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE-EXT_SOURCE_1_mean max value =  0.549981327597728
Calculating mean of  EXT_SOURCE_2  by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE-EXT_SOU

In [189]:
# train_features = pd.DataFrame(np.concatenate([df[count_columns][0:n_train].values, train_stacked.values, df[my_features][0:n_train].values, goran_features[0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values], axis=1), columns=
#                             count_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns))
# test_features = pd.DataFrame(np.concatenate([df[count_columns][n_train:].values, test_stacked.values, df[my_features][n_train:].values, goran_features[n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values], axis=1), columns=
#                            count_columns +  ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns))

In [190]:
# train_features = np.concatenate([train_stacked.values, df[my_features][0:n_train].values, goran_features[0:n_train].values, suresh_august16[:n_train].values], axis=1)
# test_features = np.concatenate([test_stacked.values, df[my_features][n_train:].values, goran_features[n_train:].values, suresh_august16[n_train:].values], axis=1)

In [191]:
# train_features = pd.DataFrame(np.concatenate([train_stacked.values, df[my_features][0:n_train].values, goran_features[0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values, suresh_august16_2[0:n_train].values], axis=1), columns=
#                              ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns) + list(suresh_august16_2.columns))
# test_features = pd.DataFrame(np.concatenate([test_stacked.values, df[my_features][n_train:].values, goran_features[n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values, suresh_august16_2[n_train:].values], axis=1), columns=
#                             ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns) + list(suresh_august16_2.columns))

In [192]:
# train_features = pd.DataFrame(np.concatenate([train_stacked.values, df[my_features][0:n_train].values, goran_features[0:n_train].values, suresh_august19[:n_train].values, suresh_august15[0:n_train].values, prevs_df[0:n_train].values, suresh_august16[0:n_train].values, suresh_august16_2[0:n_train].values], axis=1), columns=
#                             ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august19.columns) + list(suresh_august15.columns) + list(prevs_df.columns) + list(suresh_august16.columns) + list(suresh_august16_2.columns))
# test_features = pd.DataFrame(np.concatenate([test_stacked.values, df[my_features][n_train:].values, goran_features[n_train:].values, suresh_august19[n_train:].values, suresh_august15[n_train:].values, prevs_df[n_train:].values, suresh_august16[n_train:].values, suresh_august16_2[n_train:].values], axis=1), columns=
#                             ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august19.columns) + list(suresh_august15.columns) + list(prevs_df.columns) +  list(suresh_august16.columns) + list(suresh_august16_2.columns))

In [193]:
# train_features = pd.DataFrame(np.concatenate([df[count_columns][0:n_train].values, train_stacked.values, df[my_features][0:n_train].values, suresh_august19[:n_train].values, suresh_august15[0:n_train].values, prevs_df[0:n_train].values, suresh_august16[0:n_train].values, suresh_august16_2[0:n_train].values], axis=1), columns=
#                             count_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features  + list(suresh_august19.columns) + list(suresh_august15.columns) + list(prevs_df.columns) + list(suresh_august16.columns) + list(suresh_august16_2.columns))
# test_features = pd.DataFrame(np.concatenate([df[count_columns][n_train:].values, test_stacked.values, df[my_features][n_train:].values, suresh_august19[n_train:].values, suresh_august15[n_train:].values, prevs_df[n_train:].values, suresh_august16[n_train:].values, suresh_august16_2[n_train:].values], axis=1), columns=
#                             count_columns + ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features + list(suresh_august19.columns) + list(suresh_august15.columns) + list(prevs_df.columns) +  list(suresh_august16.columns) + list(suresh_august16_2.columns))

In [212]:
new_df[mean_columns][0:n_train].values

array([[28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0],
       [16278, 61106, 35477, ...,     0,     0,     0],
       ...,
       [28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0]], dtype=uint16)

In [213]:
new_df[mean_columns][n_train:].values

array([[28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0],
       ...,
       [28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0],
       [28439, 22017, 39368, ...,     0,     0,     0]], dtype=uint16)

In [195]:
gc.collect()

0

In [67]:
# train_features = pd.DataFrame(np.concatenate([new_df[mean_columns][0:n_train].values, suresh_august16[0:n_train].values, df[count_columns][0:n_train].values , df[counts_columns][0:n_train].values,  train_stacked.values, df[my_features][0:n_train].values], axis=1), columns=
#                             mean_columns +  list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features)
# test_features = pd.DataFrame(np.concatenate([new_df[mean_columns][n_train:].values, suresh_august16[n_train:].values, df[count_columns][n_train:].values, df[counts_columns][n_train:].values,  test_stacked.values, df[my_features][n_train:].values], axis=1), columns=
#                              mean_columns + list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features)

In [68]:
# train_features = pd.DataFrame(np.concatenate([suresh_august16[0:n_train].values, df[count_columns][0:n_train].values , df[counts_columns][0:n_train].values,  train_stacked.values, df[my_features][0:n_train].values], axis=1), columns=
#                             list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features)
# test_features = pd.DataFrame(np.concatenate([ suresh_august16[n_train:].values, df[count_columns][n_train:].values, df[counts_columns][n_train:].values,  test_stacked.values, df[my_features][n_train:].values], axis=1), columns=
#                                list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features)

In [69]:
# train_features = pd.DataFrame(np.concatenate([df[categorical_columns][0:n_train].values, goran_features_19_8[0:n_train].values, suresh_august16[0:n_train].values, df[count_columns][0:n_train].values , df[counts_columns][0:n_train].values,  train_stacked.values, df[my_features][0:n_train].values], axis=1), columns=
#                            categorical_columns + list(goran_features_19_8.columns) + list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features)
# test_features = pd.DataFrame(np.concatenate([df[categorical_columns][n_train:].values, goran_features_19_8[n_train:].values, suresh_august16[n_train:].values, df[count_columns][n_train:].values, df[counts_columns][n_train:].values,  test_stacked.values, df[my_features][n_train:].values], axis=1), columns=
#                              categorical_columns + list(goran_features_19_8.columns) +  list(suresh_august16.columns) + count_columns + counts_columns + ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features)

In [70]:
# train_features = pd.DataFrame(np.concatenate([goranm_8_20[0:n_train].values ,goran_features_19_8[0:n_train].values, suresh_august20[0:n_train].values, train_stacked.values, df[my_features][0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values], axis=1), columns=
#               list(goranm_8_20.columns) + list(goran_features_19_8.columns) +  list(suresh_august20.columns) + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features + list(suresh_august16.columns) + list(suresh_august15.columns))
# test_features = pd.DataFrame(np.concatenate([goranm_8_20[n_train:].values, goran_features_19_8[n_train:].values, suresh_august20[n_train:].values, test_stacked.values, df[my_features][n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values], axis=1), columns=
#               list(goranm_8_20.columns) + list(goran_features_19_8.columns)  + list(suresh_august20.columns) + ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features  + list(suresh_august16.columns) + list(suresh_august15.columns))

In [74]:
# train_features = pd.DataFrame(np.concatenate([goranm_8_20[0:n_train].values ,goran_features_19_8[0:n_train].values, suresh_august20[0:n_train].values, train_stacked.iloc[:, selected_features].values, df[my_features][0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values], axis=1), columns=
#               list(goranm_8_20.columns) + list(goran_features_19_8.columns) +  list(suresh_august20.columns) + ['y_' + str(i) for i in selected_features] + my_features + list(suresh_august16.columns) + list(suresh_august15.columns))
# test_features = pd.DataFrame(np.concatenate([goranm_8_20[n_train:].values, goran_features_19_8[n_train:].values, suresh_august20[n_train:].values, test_stacked.iloc[:, selected_features].values, df[my_features][n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values], axis=1), columns=
#               list(goranm_8_20.columns) + list(goran_features_19_8.columns)  + list(suresh_august20.columns) + ['y_' + str(i) for i in selected_features] + my_features  + list(suresh_august16.columns) + list(suresh_august15.columns))

In [75]:
# train_features = pd.DataFrame(np.concatenate([goran_features_19_8[0:n_train].values, df[count_columns][0:n_train].values, train_stacked.values, df[my_features][0:n_train].values, goran_features[0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values], axis=1), columns=
#                           list(goran_features_19_8.columns) +   count_columns + ['y_' + str(i) for i in range(train_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns))
# test_features = pd.DataFrame(np.concatenate([goran_features_19_8[n_train:].values, df[count_columns][n_train:].values, test_stacked.values, df[my_features][n_train:].values, goran_features[n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values], axis=1), columns=
#                            list(goran_features_19_8.columns) + count_columns +  ['y_' + str(i) for i in range(test_stacked.shape[1])] + my_features + list(goran_features.columns) + list(suresh_august16.columns) + list(suresh_august15.columns))

In [217]:
train_features = pd.DataFrame(np.concatenate([df[counts_columns][0:n_train].values, df[count_columns][0:n_train].values ,new_df[mean_columns][0:n_train].values, prevs_df[0:n_train].values, suresh_20[0:n_train].values, goranm_8_20[0:n_train].values ,goran_features_19_8[0:n_train].values, suresh_august20[0:n_train].values,  df[my_features][0:n_train].values, suresh_august16[:n_train].values, suresh_august15[0:n_train].values], axis=1), columns=
           counts_columns + count_columns + mean_columns +  list(prevs_df.columns) + list(suresh_20.columns) + list(goranm_8_20.columns) + list(goran_features_19_8.columns) +  list(suresh_august20.columns) + my_features + list(suresh_august16.columns) + list(suresh_august15.columns))
test_features = pd.DataFrame(np.concatenate([df[counts_columns][n_train:].values, df[count_columns][n_train:].values, new_df[mean_columns][n_train:].values, prevs_df[n_train:].values, suresh_20[n_train:].values, goranm_8_20[n_train:].values, goran_features_19_8[n_train:].values, suresh_august20[n_train:].values, df[my_features][n_train:].values, suresh_august16[n_train:].values, suresh_august15[n_train:].values], axis=1), columns=
           counts_columns + count_columns + mean_columns + list(prevs_df.columns) +   list(suresh_20.columns) + list(goranm_8_20.columns) + list(goran_features_19_8.columns)  + list(suresh_august20.columns)  + my_features  + list(suresh_august16.columns) + list(suresh_august15.columns))

In [218]:
test_features.head()

,NAME_CONTRACT_TYPE-CODE_GENDER_cunique,NAME_CONTRACT_TYPE-FLAG_OWN_CAR_cunique,NAME_CONTRACT_TYPE-FLAG_OWN_REALTY_cunique,NAME_CONTRACT_TYPE-NAME_TYPE_SUITE_cunique,NAME_CONTRACT_TYPE-NAME_INCOME_TYPE_cunique,NAME_CONTRACT_TYPE-NAME_EDUCATION_TYPE_cunique,NAME_CONTRACT_TYPE-NAME_FAMILY_STATUS_cunique,NAME_CONTRACT_TYPE-NAME_HOUSING_TYPE_cunique,NAME_CONTRACT_TYPE-OCCUPATION_TYPE_cunique,NAME_CONTRACT_TYPE-WEEKDAY_APPR_PROCESS_START_cunique,...,AMT_INCOME_TOTAL_12_AMT_ANNUITY_2_SURESH,TOTAL_CC_LOADING_6MONTHS_SURESH,INSTALLMENT_PAID_LATE_IN_DAYS_12_MONTHS_SUM_SURESH,COUNT_SCOFR_SURESH,INSTALLMENT_PAID_OVER_AMOUNT_LAST_LOAN_MEAN_SURESH,CREDIT_PER_NON_CHILD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SUM_SURESH,MEAN_PAYMENT_CC_SURESH,DAYS_DECISION_MAX_SURESH
0,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,284400.0,13.820275,-62.0,NaN,-1740.0
1,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,111384.0,13.510284,-212.0,NaN,-315.0
2,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,-52902.0000,0.000000,-13.0,NaN,-69.078947,331632.0,7.726018,-370.0,-8.526513e-14,-222.0
3,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,-32682.4875,0.264373,-12.0,NaN,-837.478929,787500.0,4.914531,-140.0,7.619574e+02,-531.0
4,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,312750.0,2.667140,-147.0,NaN,-111.0


In [219]:
gc.collect()

3270

In [220]:
cols_to_drop = [
    
  'STCK_BERBAL_6_.', 
  "FLAG_DOCUMENT_2",

  "FLAG_DOCUMENT_7",

  "FLAG_DOCUMENT_10",

  "FLAG_DOCUMENT_12",

  "FLAG_DOCUMENT_13",

  "FLAG_DOCUMENT_14",

  "FLAG_DOCUMENT_15",

  "FLAG_DOCUMENT_16",

  "FLAG_DOCUMENT_17",

  "FLAG_DOCUMENT_18",

  "FLAG_DOCUMENT_19",

  "FLAG_DOCUMENT_20",

  "FLAG_DOCUMENT_21",

  "PREV_NAME_CONTRACT_TYPE_Consumer_loans",

  "PREV_NAME_CONTRACT_TYPE_XNA",

  "PB_CNT_NAME_CONTRACT_STATUS_Amortized_debt",

  "MAX_DATA_ALL",

  "MIN_DATA_ALL",

  "MAX_MIN_DURATION",

  "MAX_AMT_CREDIT_MAX_OVERDUE",

  "CC_AMT_DRAWINGS_ATM_CURRENT_MIN",

  "CC_AMT_DRAWINGS_OTHER_CURRENT_MAX",

  "CC_AMT_DRAWINGS_OTHER_CURRENT_MIN",

  "CC_CNT_DRAWINGS_ATM_CURRENT_MIN",

  "CC_CNT_DRAWINGS_OTHER_CURRENT_MAX",

  "CC_CNT_DRAWINGS_OTHER_CURRENT_MIN",

  "CC_SK_DPD_DEF_MIN",

  "CC_SK_DPD_MIN",

  "BERB_STATUS_CREDIT_TYPE_Loan_for_working_capital_replenishment",

 "BERB_STATUS_CREDIT_TYPE_Real_estate_loan",

  "BERB_STATUS_CREDIT_TYPE_Loan_for_the_purchase_of_equipment",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Loan_for_working_capital_replenishmentClosed",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Car_loanSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Another_type_of_loanActive",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Loan_for_working_capital_replenishmentSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_MicroloanSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Another_type_of_loanSold",

  "FLAG_EMAIL",

  "APARTMENTS_AVG",

  "AMT_REQ_CREDIT_BUREAU_MON",

  "AMT_REQ_CREDIT_BUREAU_QRT",

  "AMT_REQ_CREDIT_BUREAU_YEAR",

  "STCK_BERBAL_6_",

  "STCK_CC_6_x"]

In [221]:
feats = [f for f in cols_to_drop if f in train_features.columns]

In [222]:
train_features.drop(labels=feats, axis=1, inplace=True)

In [223]:
test_features.drop(labels=feats, axis=1, inplace=True)

In [224]:
cat_features = [] # [i for i in range(len(categorical_columns))]

In [225]:
gc.collect()

7

In [226]:
# train_stacked.to_csv('oofs/train_oofs-v0.1.0.csv', index=False)
# test_stacked.to_csv('oofs/test_oofs-v0.1.0.csv', index=False)

In [227]:
test_features.head()

,NAME_CONTRACT_TYPE-CODE_GENDER_cunique,NAME_CONTRACT_TYPE-FLAG_OWN_CAR_cunique,NAME_CONTRACT_TYPE-FLAG_OWN_REALTY_cunique,NAME_CONTRACT_TYPE-NAME_TYPE_SUITE_cunique,NAME_CONTRACT_TYPE-NAME_INCOME_TYPE_cunique,NAME_CONTRACT_TYPE-NAME_EDUCATION_TYPE_cunique,NAME_CONTRACT_TYPE-NAME_FAMILY_STATUS_cunique,NAME_CONTRACT_TYPE-NAME_HOUSING_TYPE_cunique,NAME_CONTRACT_TYPE-OCCUPATION_TYPE_cunique,NAME_CONTRACT_TYPE-WEEKDAY_APPR_PROCESS_START_cunique,...,AMT_INCOME_TOTAL_12_AMT_ANNUITY_2_SURESH,TOTAL_CC_LOADING_6MONTHS_SURESH,INSTALLMENT_PAID_LATE_IN_DAYS_12_MONTHS_SUM_SURESH,COUNT_SCOFR_SURESH,INSTALLMENT_PAID_OVER_AMOUNT_LAST_LOAN_MEAN_SURESH,CREDIT_PER_NON_CHILD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SD_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_SUM_SURESH,MEAN_PAYMENT_CC_SURESH,DAYS_DECISION_MAX_SURESH
0,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,284400.0,13.820275,-62.0,NaN,-1740.0
1,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,111384.0,13.510284,-212.0,NaN,-315.0
2,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,-52902.0000,0.000000,-13.0,NaN,-69.078947,331632.0,7.726018,-370.0,-8.526513e-14,-222.0
3,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,-32682.4875,0.264373,-12.0,NaN,-837.478929,787500.0,4.914531,-140.0,7.619574e+02,-531.0
4,2.0,2.0,2.0,8.0,7.0,5.0,5.0,6.0,19.0,7.0,...,NaN,NaN,NaN,NaN,0.000000,312750.0,2.667140,-147.0,NaN,-111.0


In [228]:
train_features['nans'] = train_features.replace([np.inf, -np.inf], np.nan).isnull().sum(axis=1)
test_features['nans'] = test_features.replace([np.inf, -np.inf], np.nan).isnull().sum(axis=1)

In [246]:
test_file_path = "Level_1_stack/test_catb_xxx_0.csv"
validation_file_path = 'Level_1_stack/validation_catb_xxx_0.csv.csv'
num_folds = 5

In [247]:
# train_features = train_features.replace([np.inf, -np.inf], np.nan).fillna(-999, inplace=False)
# test_features = test_features.replace([np.inf, -np.inf], np.nan).fillna(-999, inplace=False)

In [ ]:
gc.collect()
encoding = 'ohe'

train_df = train_features
test_df = test_features

print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()
# Cross validation model
folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

#feats = [col for col in feats_0 if df[col].dtype == 'object']


print(train_df[feats].shape)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train['TARGET'])):
        
      
    
        if encoding == 'ohe':
            x_train = train_df[feats].iloc[train_idx]
            #cat_features = [i for i, col in enumerate(x_train.columns) if col in categorical_cols]
            x_train = x_train.replace([np.inf, -np.inf], np.nan).fillna(-999).values
            x_valid = train_df[feats].iloc[valid_idx].replace([np.inf, -np.inf], np.nan).fillna(-999).values
            x_test = test_df[feats].replace([np.inf, -np.inf], np.nan).fillna(-999).values
            print(x_train.shape, x_valid.shape, x_test.shape)
            
            gc.collect()
            
        clf = CatBoostRegressor(learning_rate=0.05, iterations=2500, verbose=True, rsm=0.25,
                              use_best_model=True, l2_leaf_reg=40, allow_writing_files=False, metric_period=50,
                              random_seed=666, depth=6, loss_function='RMSE', od_wait=50, od_type='Iter')

        clf.fit(x_train, train['TARGET'].iloc[train_idx].values, eval_set=(x_valid, train['TARGET'].iloc[valid_idx].values)
                    , cat_features=[], use_best_model=True, verbose=True)

        oof_preds[valid_idx] = clf.predict(x_valid)
        sub_preds += clf.predict(x_test) / folds.n_splits

        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(train['TARGET'].iloc[valid_idx].values, oof_preds[valid_idx])))
        del clf
        gc.collect()
    
        
        



Starting LightGBM. Train shape: (307511, 423), test shape: (48744, 423)
(307511, 421)
(246008, 421) (61503, 421) (48744, 421)
0:	learn: 0.2813103	test: 0.2859832	best: 0.2859832 (0)	total: 91.9ms	remaining: 3m 49s
50:	learn: 0.2600347	test: 0.2638970	best: 0.2638970 (50)	total: 4.76s	remaining: 3m 48s
100:	learn: 0.2582922	test: 0.2622716	best: 0.2622716 (100)	total: 9.58s	remaining: 3m 47s
150:	learn: 0.2574201	test: 0.2616139	best: 0.2616139 (150)	total: 14.3s	remaining: 3m 42s
200:	learn: 0.2568222	test: 0.2612286	best: 0.2612286 (200)	total: 19s	remaining: 3m 37s
250:	learn: 0.2563299	test: 0.2609930	best: 0.2609930 (250)	total: 23.7s	remaining: 3m 32s
300:	learn: 0.2559197	test: 0.2608294	best: 0.2608294 (300)	total: 28.5s	remaining: 3m 27s
350:	learn: 0.2555578	test: 0.2606731	best: 0.2606731 (350)	total: 33.2s	remaining: 3m 23s
400:	learn: 0.2552333	test: 0.2605597	best: 0.2605597 (400)	total: 38s	remaining: 3m 18s
450:	learn: 0.2549128	test: 0.2604635	best: 0.2604635 (450)	tota

Fold  3 AUC : 0.778853
(246009, 421) (61502, 421) (48744, 421)
0:	learn: 0.2823437	test: 0.2821966	best: 0.2821966 (0)	total: 90.3ms	remaining: 3m 45s
50:	learn: 0.2606358	test: 0.2611654	best: 0.2611654 (50)	total: 4.82s	remaining: 3m 51s
100:	learn: 0.2588962	test: 0.2597523	best: 0.2597523 (100)	total: 9.62s	remaining: 3m 48s
150:	learn: 0.2580271	test: 0.2591569	best: 0.2591569 (150)	total: 14.4s	remaining: 3m 44s
200:	learn: 0.2573996	test: 0.2587936	best: 0.2587936 (200)	total: 19.1s	remaining: 3m 38s
250:	learn: 0.2569301	test: 0.2585362	best: 0.2585362 (250)	total: 23.9s	remaining: 3m 34s
300:	learn: 0.2565027	test: 0.2583558	best: 0.2583558 (300)	total: 28.7s	remaining: 3m 29s
350:	learn: 0.2561332	test: 0.2582071	best: 0.2582071 (350)	total: 33.4s	remaining: 3m 24s
400:	learn: 0.2558082	test: 0.2580959	best: 0.2580959 (400)	total: 38.1s	remaining: 3m 19s
450:	learn: 0.2555063	test: 0.2580028	best: 0.2580028 (450)	total: 42.9s	remaining: 3m 15s
500:	learn: 0.2552172	test: 0.25

In [ ]:
sub_df = test[['SK_ID_CURR']].copy()
sub_df['TARGET'] = sub_preds
sub_df[['SK_ID_CURR', 'TARGET']].to_csv(test_file_path, index= False)


val_df = train[['SK_ID_CURR', 'TARGET']].copy()
val_df['TARGET'] = oof_preds
val_df[['SK_ID_CURR', 'TARGET']].to_csv(validation_file_path, index= False)        

gc.collect()